In [1]:
import numpy as np
import torch
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append('../')

from src.features import Q_measure, dist_distribution, convex_hull
from src.andi_tools import Andi_to_xy, Data_to_xy
from src.training import FeedForward

%load_ext autoreload
%autoreload 2

In [2]:
trajectories, labels = Data_to_xy("../data_generation/test_2d.csv","../data_generation/label_2d.csv")
labels = torch.FloatTensor(labels)
labels = torch.max(labels, 1)[1]

In [3]:
len(labels)

2000

In [4]:
features =[Q_measure(trajectories, 2),
            Q_measure(trajectories, 7),
           convex_hull(trajectories,5),
          dist_distribution(trajectories)]
feature_sizes = [f.shape[1] for f in features]
features = [torch.FloatTensor(f) for f in features]

In [13]:
torch.max(features[0])

tensor(1.)

In [5]:
model = FeedForward(feature_sizes, hidden_size=32)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [6]:
model.train()
epoch = 5000
for epoch in range(epoch):    
    optimizer.zero_grad()    # Forward pass
    y_pred = model(features)    # Compute Loss
    loss = criterion(y_pred.squeeze(), labels)
    if epoch % 500 == 0 :
        print('Epoch {}: train loss: {}'.format(epoch, loss.item()))    # Backward pass
    loss.backward()
    optimizer.step()

Epoch 0: train loss: 1.6106597185134888
Epoch 500: train loss: 1.2364540100097656
Epoch 1000: train loss: 1.2107945680618286
Epoch 1500: train loss: 1.2019116878509521
Epoch 2000: train loss: 1.1972025632858276
Epoch 2500: train loss: 1.1944068670272827
Epoch 3000: train loss: 1.1920613050460815
Epoch 3500: train loss: 1.1902049779891968
Epoch 4000: train loss: 1.188753604888916
Epoch 4500: train loss: 1.1878554821014404


In [7]:
after_train = criterion(y_pred,labels)

In [8]:
prediction = torch.max(y_pred, 1)[1]

In [9]:
nonzero = np.count_nonzero(np.argmax(y_pred.detach().numpy(),axis=1)-labels.detach().numpy())
tot = len(np.argmax(y_pred.detach().numpy(),axis=1)-labels.detach().numpy())

In [10]:
(tot-nonzero)/tot

0.719